In [64]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup 
from urllib.request import urlopen
import numpy as np
import pandas as pd

In [65]:
global manfac
global country
manfac = { '현대' : 16, '기아' : 12, '제네시스' : 61321, '쌍용' : 13, '르노삼성' : 15, '한국지엠' : 14,
          '폭스바겐' : 20, '벤츠' : 21, 'BMW' : 23, '아우디' : 18, '렉서스' : 22, '도요타' : 41, '혼다' : 31, '볼보' : 24, '미니' : 6435, 
          '재규어' : 25, '랜드로버' : 39, '포드' : 30, '링컨' : 35, '쉐보레' : 48, '캐딜락' : 19, '테슬라' : 29832, '지프' : 40, 
          '르노' : 3847, '푸조' : 28, '시트로엥' : 3848, 'DS' : 128687, '마세라티' : 3976, '포르쉐' : 3905,
          '람보르기니' : 3814, '페라리' : 27, '맥라렌' : 29978, '부가티' : 58745, '애스턴마틴' : 3827, '마이바흐' : 6434,
          '롤스로이스' : 3824, '벤틀리' : 4129, '닛산' : 33, '로터스' : 4040, '마쯔다' : 42, '미쓰비스' : 37, '복스홀' : 29989, 
          '뷰익' : 43, '사이언' : 4216, '세아트' : 29984, '스마트' : 6436, '인피니티' : 34, '코닉세그' : 3840, '알파로메오' : 3999,
          '크라이슬러' : 26, 'GMC' : 29611, '닷지' : 4188}
country = {'국산' : 'N',
          '수입' : 'Y'}

In [66]:
global imp
global manf

# 국산인지 수입인지 입력
con = '수입'
# 제조사 입력
mf = '포르쉐'

In [67]:
def getUrl(ctry,manf_name,page):
    url_base = 'https://auto.naver.com/'
    manf_no = manfac[manf_name]
    con = country[ctry]
    url_sub = 'company/main.nhn?mnfcoNo='+str(manf_no)+'&modelType=DC&order=3&importYn='+con+'&lnchYY=-1&saleType=-1&page='+str(page)
    url = url_base + url_sub

    html = urlopen(url)
    bs = BeautifulSoup(html, "html.parser")
    return bs

In [68]:
test_bs = getUrl(con,mf,1)

In [69]:
def getName(bs,length):
    new_name = bs.select('div.sum > a.model_name > span.box > strong')
    new_name_year = np.array([],dtype=np.int8)
    new_name_model = np.array([])

    if country[con] == 'N':
        for i in np.arange(0,length):
            new_name_year = np.append(new_name_year,new_name[i].text.split(' ',1)[0])
            new_name_model = np.append(new_name_model,new_name[i].text.split(' ',1)[1])
    
    elif country[con] == 'Y':
        for i in np.arange(0,length):
            new_name_year = np.append(new_name_year,new_name[i].text.split(' ',2)[0])
            new_name_model = np.append(new_name_model,new_name[i].text.split(' ',2)[2])
    return new_name_year,new_name_model

In [70]:
getName(test_bs,15)

(array(['2007', '2016', '2006', '2018', '2015', '2013', '2012', '2009',
        '2017', '2015', '2015', '2014', '2011', '2017', '2020'],
       dtype='<U4'),
 array(['박스터', '마칸 GTS', '카이맨', '마칸', '마칸', '박스터', '박스터', '박스터', '카이맨',
        '카이맨', '박스터', '카이맨 ', '박스터', '마칸', '타이칸'], dtype='<U32'))

In [71]:
def getPrice(bs):
    new_price = bs.select('ul.lst > li.price.new > span.cont')
    length = len(new_price)
    new_price_min = np.array([],dtype=np.int8)
    new_price_max = np.array([],dtype=np.int8)
    for i in np.arange(0,len(new_price)):
        min_p = int(new_price[i].text.split('~')[0].replace('억','').replace(',',''))
        
        if len(new_price[i].text.split('~')) == 1:
            max_p = int(new_price[i].text.split('~')[0].replace('억','').replace(',',''))
        else:
            max_p = int(new_price[i].text.split('~')[1].replace('억','').replace(',',''))
        
        if min_p > max_p:
            max_p = max_p + 9000
        
        new_price_min = np.append(new_price_min,min_p)
        if len(new_price[i].text.split('~')) == 1:
            new_price_max = np.append(new_price_max,min_p)
        else:
            new_price_max = np.append(new_price_max,max_p)
        
        
            
    return new_price_min,new_price_max,length

In [72]:
getPrice(test_bs)

(array([ 6419, 67200,  6972,  7560,  7560,  7750,  7750,  7791,  7800,
         7840,  7840,  7930,  8150,  8240, 79900]),
 array([  7594,  67200,  11100,   7560,  10800,  11120,   9890,   9686,
          9400,   9400,   9400,  11480,  11450,  10800, 195870]),
 15)

In [73]:
def getMileage(bs,length):
    new_mileage = bs.select('ul.lst > li.mileage > span.dt > span.ell > span.en')
    new_mileage_min = np.array([])
    new_mileage_max = np.array([])
    for i in np.arange(0,length):
        new_mileage_min = np.append(new_mileage_min,new_mileage[i].text.split('~')[0].replace('km/ℓ','').replace('km/kg',''))
        if len(new_mileage[i].text.split('~')) == 1:
            new_mileage_max = np.append(new_mileage_max,new_mileage[i].text.split('~')[0].replace('km/ℓ','').replace('km/kg',''))
        else:
            new_mileage_max = np.append(new_mileage_max,new_mileage[i].text.split('~')[1].replace('km/ℓ','').replace('km/kg',''))
    return new_mileage_min,new_mileage_max

In [74]:
getMileage(test_bs,10)

(array(['7.8', '9.2ℓ/100km', '9.3', '9.0', '8.1', '9.4', '9.5', '10.2',
        '9.0', '10.1'], dtype='<U32'),
 array(['9.1', '9.2ℓ/100km', '9.8', '9.0', '16.4', '10.1', '10.7', '11.0',
        '9.4', '10.7'], dtype='<U32'))

In [75]:
def getFuel(bs,length):
    new_fuel = bs.select('ul.lst > li.mileage > span.dt > span.ell')
    new_fuel_arr = np.array([])
    for i in np.arange(1,length*2,2):
        new_fuel_arr = np.append(new_fuel_arr,new_fuel[i].text.replace('\r','').replace('\n','').replace('\t',''))
    return new_fuel_arr

In [76]:
getFuel(test_bs,10)

array(['가솔린', '가솔린', '가솔린', '가솔린', '디젤, 가솔린', '가솔린', '가솔린', '가솔린', '가솔린',
       '가솔린'], dtype='<U32')

In [77]:
def getSize(bs,length):
    new_size = bs.select('ul.lst > li.info > a.type > span.cont')
    new_size_arr = np.array([])
    for i in np.arange(0,length):
        new_size_arr = np.append(new_size_arr,new_size[i].text)
    return new_size_arr

In [78]:
getSize(test_bs,10)

array(['스포츠카', '중형', '스포츠카', '중형', '중형', '스포츠카', '스포츠카', '스포츠카', '스포츠카',
       '스포츠카'], dtype='<U32')

In [79]:
def getNewInfo(con,mf):
    re_name_year = np.array([])
    re_name_model = np.array([])
    re_price_min = np.array([])
    re_price_max = np.array([])
    re_mileage_min = np.array([])
    re_mileage_max = np.array([])
    re_fuel = np.array([])
    re_size = np.array([])
    
    for i in np.arange(1,23):
        bs = getUrl(con,mf,i)
        length = getPrice(bs)[2]
        re_name_year = np.append(re_name_year,getName(bs,length)[0])
        re_name_model = np.append(re_name_model,getName(bs,length)[1])
        re_price_min = np.append(re_price_min,getPrice(bs)[0])
        re_price_max = np.append(re_price_max,getPrice(bs)[1])
        re_mileage_min = np.append(re_mileage_min,getMileage(bs,length)[0])
        re_mileage_max = np.append(re_mileage_max,getMileage(bs,length)[1])
        re_fuel = np.append(re_fuel,getFuel(bs,length))
        re_size = np.append(re_size,getSize(bs,length))
        if length < 15:
            break;
    new_data = pd.DataFrame({'Release Year' : re_name_year,
                        'Model Name' : re_name_model,
                        'Min Price(won)' : re_price_min,
                        'Max Price(won)' : re_price_max,
                        'Min Mileage' : re_mileage_min,
                        'Max Mileage' : re_mileage_max,
                         'Fuel' : re_fuel,
                        'Size' : re_size})
    return new_data

In [80]:
#제조사별 번호를 대입한다.
test = getNewInfo(con,mf)

In [81]:
test = test.sort_values(by='Release Year',ascending=False)

In [82]:
test = test.set_index('Release Year')

In [83]:
#연식별 차종 가격 비교

In [84]:
test

,Model Name,Min Price(won),Max Price(won),Min Mileage,Max Mileage,Fuel,Size
Release Year,,,,,,,
2021,타이칸 크로스 투리스모,90900.0,187600.0,3.8,3.9km/kWh,전기,준대형
2021,911 GT3,152416.0,152416.0,12.4,13.3ℓ/100km,가솔린,스포츠카
2020,타이칸,79900.0,195870.0,2.7km/kWh,2.7km/kWh,전기,준대형
2020,마칸,91922.0,91922.0,9.8ℓ/100km,9.8ℓ/100km,가솔린,중형
2020,박스터 GTS,83949.0,83949.0,정보없음,정보없음,가솔린,스포츠카
...,...,...,...,...,...,...,...
2007,박스터,6419.0,7594.0,7.8,9.1,가솔린,스포츠카
2006,카이엔,9990.0,17400.0,5.3,6.2,가솔린,준대형
2006,카이맨,6972.0,11100.0,9.3,9.8,가솔린,스포츠카


In [85]:
#test.to_csv('%s_new_car_list.csv'%(manf))